# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
import warnings
from constants import *
import math
import os
import random
import matplotlib.pyplot as plt
import dill
from functools import reduce

# Helper Methods

In [2]:
def generate_random_sample(X, size):
    '''
    Given a list X, 
    generate random samples of given size
    '''
    Z_temp = random.sample(list(X), size)
    
    #Concatenation
    Z = [Z_temp[0]]
    for val in Z_temp[1:]:
        Z = np.concatenate([Z,[val]])
    
    return Z

def convert_values_to_list(list_val):
    '''
    Given a list X = [1 2 3] , 
    return X = [[1],[2],[3]]
    '''
    X = []
    for i in list_val:
        X.append([i])
    return X

def conversion_list_of_list(X, DIMENSION):
    '''
    Given a list X with values in lists,
    X = [[ 3  2  2],[4 8 10]]
    Convert each value to a list
    Return list of lists, array of lengths of each sequence
    X = [[3] [2] [2] [4] [8] [10]]
    ''' 
    X_new = []
    length = len(X)
    for idx, val_list in enumerate(X):
        Y = []
        for val in val_list:
            Y.append([val])
        X_new.append(Y)

    #Concatenation
    Z = X_new[0]
    for val_list in X_new[1:]:
        Z = np.concatenate([Z,val_list])

    # assign array of lengths for HMM
    lengths = [DIMENSION]*length
    
    return Z,lengths

#Calculate likelihood for given sequence according to given HMMs and return HMM
def likelihood_sequence(sequence, HMM_array):
    '''
    Given list of K HMMs and sequence,
    determines likelihood of sequence under all HMM models
    Returns index of HMM which has max likelihood
    ''' 
    scores = []
    length = [len(sequence)]
    for i, HMM in enumerate(HMM_array):
        calculated_score = HMM.score(sequence, length)
        scores.append(calculated_score)
    idx = scores.index(max(scores))
    return idx

def HMM_model_stats(model):
    '''
    Details of HMM model
    ''' 
    print("*************************************")
    print("Transition matrix")
    print(model.transmat_)
    print("*************************************")
    print("Means and stds of each hidden state")
    for i in range(model.n_components):
        print("Hidden state {0}".format(i))
        print("mean = ", model.means_[i])
        print("std = ", [np.sqrt(model.covars_[i])])
        print()

def BIC(model,X,lengths):
    LogLikelihood = model.score(X,lengths)
    num_hidden_states = model.n_components
    # D counts transition matrix (emission estimated by PDF), means = num_hidden_states  
    # covariance matrix = num_hidden_states
    D = num_hidden_states**2-num_hidden_states + 2*num_hidden_states
    BIC = LogLikelihood - (D/2)*np.log(len(X))
    return BIC
    
def BIC_array(HMM_array,X_i):
    BIC_total = 0
    for i in range(len(X_i)):
        model = HMM_array[i]
        X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        BIC_total+= BIC(model,X,lengths)
    return BIC_total

def likelihood_array(HMM_array,X_i):
    likelihood_total = 0
    for i in range(len(X_i)):
        if(len(X_i[i])>=HMM_array[i].n_components):
            model = HMM_array[i]
            X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
            LogLikelihood = model.score(X,lengths)
            likelihood = LogLikelihood 
            likelihood_total+= likelihood
    return likelihood_total

def plot_BIC(list_k, BIC_score):
    fig = plt.subplot(111)
    plt.plot(list_k, BIC_score, marker='o')  
    plt.xlabel('Value of K')
    plt.ylabel('Objective')
    plt.title('BIC')
    plt.show() 

def print_stats(assignments,length):
    for i in range(length):
        if(i%100==0):
            print(assignments[i])   

# Merge data

In [ ]:
previous = 'RPKMOutput/GeneNames/RNASeq/geneTXCD_RPKMoutput_'
timepoints = ['ATCACG-s_6_1_genome.txt','TTAGGC-s_6_1_genome.txt',
             'CAGATC-s_6_1_genome.txt','GCCAAT-s_6_1_genome.txt',
             'CTTGTA-s_6_1_genome.txt'
            ]
dataframes = []
for idx,file in enumerate(timepoints):
    df = pd.read_csv(previous+file, sep="	", na_values=['-'])
    df = df.dropna(how='any')
    df = df[['AccNum', 'GeneName','cdReads',
       'cdRPKM']]
    df.rename(columns={'cdRPKM':'cdRPKM'+str(idx), 'cdReads':'cdReads'+str(idx)}, inplace=True)
    dataframes.append(df)

In [ ]:
df_rnaseq = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), dataframes)

In [ ]:
df_rnaseq.head()

In [ ]:
# df_rnaseq.to_csv('RPKMOutput/RNASeq.txt',sep=' ',index=False)

In [ ]:
previous = 'RPKMOutput/GeneNames/RPF/geneTXCD_RPKMoutput_'
timepoints = ['ATCACG-s_7_1_genome.txt','TTAGGC-s_7_1_genome.txt',
             'CAGATC-s_7_1_genome.txt','GCCAAT-s_7_1_genome.txt',
             'CTTGTA-s_7_1_genome.txt'
            ]
dataframes = []
for idx,file in enumerate(timepoints):
    df = pd.read_csv(previous+file, sep="	", na_values=['-'])
    df = df.dropna(how='any')
    df = df[['AccNum', 'GeneName','cdReads',
       'cdRPKM']]
    df.rename(columns={'cdRPKM':'cdRPKM'+str(idx), 'cdReads':'cdReads'+str(idx)}, inplace=True)
    dataframes.append(df)
df_rpf = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), dataframes)
df_rpf.head()

In [ ]:
# df_rpf.to_csv('RPKMOutput/RPF.txt',sep=' ',index=False)

In [ ]:
# df_rnaseq = df_rnaseq[['AccNum','GeneName', 'cdRPKM0', 'cdRPKM1',
#        'cdRPKM2', 'cdRPKM3', 'cdRPKM4']]
# df_rpf = df_rpf[['AccNum','GeneName', 'cdRPKM0', 'cdRPKM1',
#        'cdRPKM2', 'cdRPKM3', 'cdRPKM4']]
df_TE = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), [df_rnaseq,df_rpf])
df_TE.head()

In [ ]:
df_TE[['cdRPKM0_x']] = df_TE[['cdRPKM0_x']].div(df_TE['cdRPKM0_y'].values,axis=0)
df_TE[['cdRPKM1_x']] = df_TE[['cdRPKM1_x']].div(df_TE['cdRPKM1_y'].values,axis=0)
df_TE[['cdRPKM2_x']] = df_TE[['cdRPKM2_x']].div(df_TE['cdRPKM2_y'].values,axis=0)
df_TE[['cdRPKM3_x']] = df_TE[['cdRPKM3_x']].div(df_TE['cdRPKM3_y'].values,axis=0)
df_TE[['cdRPKM4_x']] = df_TE[['cdRPKM4_x']].div(df_TE['cdRPKM4_y'].values,axis=0)
for i in range(0,5):
    df_TE.rename(columns={'cdRPKM'+str(i)+'_x':'TE'+str(i)}, inplace=True)


In [ ]:
df_TE.keys()

In [ ]:
df_TE = df_TE[['AccNum', 'GeneName','TE0','TE1','TE2','TE3','TE4']]
df_TE.head()

In [ ]:
# df_TE.to_csv('RPKMOutput/TE.txt',sep=' ',index=False)

# Load Data and Clean

In [ ]:
df_rnaseq.keys()

In [4]:
#Dataset
df_rnaseq = pd.read_csv('RPKMOutput/RNASeq.txt',sep='\t')
df_rnaseq = df_rnaseq[(df_rnaseq['cdReads0'] >= 10) & (df_rnaseq['cdReads1'] >= 10) & (df_rnaseq['cdReads2'] >= 10)& (df_rnaseq['cdReads3'] >= 10)& (df_rnaseq['cdReads4'] >= 10)]
df_main = df_rnaseq[['cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']]
LENGTH,DIMENSION = df_main.shape
print("Dataset size is",LENGTH)
print("Features are", DIMENSION)
print(df_main.head(5))
X = np.log2(df_main.values)
print("****************************")
print("First 5 log2 values\n",X[:5])

Dataset size is 10340
Features are 5
     cdRPKM0    cdRPKM1    cdRPKM2    cdRPKM3    cdRPKM4
0   6.642349   5.591876   6.004645   5.369988   8.507454
2   8.115351   9.879314   9.263817   7.164127   7.730539
3  54.937502  55.371390  64.913315  67.172940  78.293873
5  49.166094  35.735701  40.321640  35.461616  35.952554
8  35.030842  32.456656  35.944257  37.945878  41.572410
****************************
First 5 log2 values
 [[ 2.7316935   2.48333227  2.58607888  2.42491875  3.08872747]
 [ 3.02065354  3.30441091  3.21160671  2.8407909   2.95056898]
 [ 5.7797194   5.79106884  6.02044253  6.06980827  6.29082751]
 [ 5.61959185  5.1592942   5.33348242  5.1481864   5.16802236]
 [ 5.13055377  5.02044247  5.16768936  5.24587127  5.37755449]]


In [5]:
df_rnaseq.head(5)

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4
0,NM_017847,ODR4,50.0,6.642349,62.0,5.591876,71.0,6.004645,30.0,5.369988,51.0,8.507454
2,NM_001003803,ATP5S,29.0,8.115351,52.0,9.879314,52.0,9.263817,19.0,7.164127,22.0,7.730539
3,NM_001003800,BICD2,778.0,54.937502,1155.0,55.371390,1444.0,64.913315,706.0,67.172940,883.0,78.293873
5,NM_016647,THEM6,170.0,49.166094,182.0,35.735701,219.0,40.321640,91.0,35.461616,99.0,35.952554
8,NM_016640,MRPS30,255.0,35.030842,348.0,32.456656,411.0,35.944257,205.0,37.945878,241.0,41.572410


# Arrays with HMM models for 1<=K<=25

In [7]:
HMM_K_ARRAYS = []
X_i_K_ARRAYS = []

# Check likelihood and do assignments

In [8]:
K_values = [13]

In [10]:
for K in K_values:
    HMM_array = []
    X_i = []
    print("**************** K =", K ,"************************")
    for i in range(K):
        X_i.append([])
        
    NUM_ITERATIONS = 0
    NUM_CLUSTER_PREV = {}
    NUM_CLUSTER_NOW = {}
    
    # Sequences for initial HMM estimation
    # Make K subsets data of LENGTH
    for i in range(LENGTH):
        for j in range(K):
            if(i%K==j):
                X_i[j].append(list(X[i]))
                NUM_CLUSTER_PREV[i] = j
                
    for i in range(K):
        model = hmm.GaussianHMM(n_components=3,covariance_type='diag')
        X_temp, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        model.fit(X_temp, lengths)
        HMM_array.append(model)
    
    likelihood_prev = likelihood_array(HMM_array,X_i)
    print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_prev)
    NUM_ITERATIONS+=1
    while (True):
        # Assign all sequences to HMM models

        print("************ Check likelihood of sequence in HMM  *********")
        NUM_CLUSTER_NOW = {}
        for idx,x in enumerate(X):
            sequence = convert_values_to_list(x)
            hmm_index = likelihood_sequence(sequence, HMM_array)
            X_i[hmm_index].append(list(x))
            NUM_CLUSTER_NOW[idx] = hmm_index
        print("************ Checking likelihood done  *********")

        # Re-estimate parameters for new HMMs
        print("************ Re-estimating HMM *********")
        HMM_array_prev = HMM_array
        HMM_array = []
        for i in range(K):
            model = hmm.GaussianHMM(n_components=3,covariance_type='diag')
            if(len(X_i[i])>=model.n_components):
                X_temp, lengths = conversion_list_of_list(X_i[i], DIMENSION)
                model.fit(X_temp, lengths)
                HMM_array.append(model)
            else:
                HMM_array.append(HMM_array_prev[i])
        print("************ Re-estimation done *********")
        likelihood_curr = likelihood_array(HMM_array,X_i)
        print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_curr)
        print("*****************************************")

        # if no reassignments, then break
        if ((NUM_CLUSTER_PREV == NUM_CLUSTER_NOW)):
            HMM_K_ARRAYS.append(HMM_array)
            X_i_K_ARRAYS.append(X_i)
            break
        else:
            # initialize empty subsets of data for next iteration
            X_i = []
            for i in range(K):
                X_i.append([])

            NUM_CLUSTER_PREV = NUM_CLUSTER_NOW
            print("Num iterations is:", NUM_ITERATIONS)
            NUM_ITERATIONS += 1
            likelihood_prev = likelihood_curr
    print("**********************************************************\n\n")

**************** K = 13 ************************
Likelihood for iteration 0 is -72711.7024899
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 1 is -107468.454663
*****************************************
Num iterations is: 1
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 2 is -14311.4695003
*****************************************
Num iterations is: 2
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 3 is -10238.9926471
*****************************************
Num iterations is: 3
******

************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 28 is -6097.17524247
*****************************************
Num iterations is: 28
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 29 is -6083.22635225
*****************************************
Num iterations is: 29
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 30 is -6361.82276842
*****************************************
Num iterations is: 30
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
****

************ Re-estimation done *********
Likelihood for iteration 55 is -5960.57588952
*****************************************
Num iterations is: 55
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 56 is -5961.5885464
*****************************************
Num iterations is: 56
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 57 is -5962.84461448
*****************************************
Num iterations is: 57
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Likelihood for iteration 58 is -5962.29349419
*******

In [11]:
print(len(HMM_K_ARRAYS))
print(len(X_i_K_ARRAYS))

1
1


In [20]:
import seaborn as sns

def plot_cluster(X,count):
    fig = plt.subplot(111)
    axes = plt.gca()
    axes.set_ylim([0,15])
    var_plot_list = ['cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']
    total=0
    for i in range(len(X)):
        fig.plot(var_plot_list, X[i])
        total+=1
    title = "HMM "+ str(count)+" : " + str(total) + " points "
    plt.title(title)
    plt.savefig('/Users/akankshitadash/Desktop/Gene Ontology/5points/RNASeq/Clusters/Cluster'+str(count)+'.png')
    plt.show()
    
def plot_heatmap(X,idx):
    plt.figure()
    sns.heatmap(X,vmin=0, vmax=10)
    plt.title('Heatmap'+str(idx))
    plt.savefig('/Users/akankshitadash/Desktop/Gene Ontology/5points/RNASeq/Heat Map/HeatMap'+str(idx+1)+'.png')
    plt.show()

In [ ]:
for idx,X in enumerate(X_i_K_ARRAYS[0]):
    plot_cluster(X,idx+1)
for idx,X in enumerate(X_i_K_ARRAYS[0]):
    plot_heatmap(X,idx)

In [22]:
df = pd.read_csv('RPKMOutput/RNASeq.txt',sep=' ')
df = df[(df['cdReads0'] >= 10) & (df['cdReads1'] >= 10) & (df['cdReads2'] >= 10)& (df['cdReads3'] >= 10)& (df['cdReads4'] >= 10)]
df = df.dropna()
df = df[['AccNum','GeneName','cdReads0','cdReads1','cdReads2','cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']]
df[['cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']] = df[['cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']].transform(lambda x: np.log2(x))
df.head(5)

,AccNum,GeneName,cdReads0,cdReads1,cdReads2,cdRPKM0,cdRPKM1,cdRPKM2,cdRPKM3,cdRPKM4
0,NM_017847,ODR4,50.0,62.0,71.0,2.731694,2.483332,2.586079,2.424919,3.088727
2,NM_001003803,ATP5S,29.0,52.0,52.0,3.020654,3.304411,3.211607,2.840791,2.950569
3,NM_001003800,BICD2,778.0,1155.0,1444.0,5.779719,5.791069,6.020443,6.069808,6.290828
5,NM_016647,THEM6,170.0,182.0,219.0,5.619592,5.159294,5.333482,5.148186,5.168022
8,NM_016640,MRPS30,255.0,348.0,411.0,5.130554,5.020442,5.167689,5.245871,5.377554


In [23]:
for idx,x in enumerate(X_i_K_ARRAYS[0]):
    genes=[]
    acc_nums=[]
    for row in x:
        temp = (df.loc[(df['cdRPKM0'] == row[0])& (df['cdRPKM1']== row[1]) & (df['cdRPKM2']== row[2])])
        if(not temp.empty):
            genes.append(temp['GeneName'].values[0])
            acc_nums.append(temp['AccNum'].values[0])
    print(len(x),len(genes))
    with open('/Users/akankshitadash/Desktop/Gene Ontology/5points/RNASeq/Gene Names/Gene'+str(idx+1)+'.txt','w') as f:
        for gene in genes:
            f.write("%s\n" % gene)
    with open('/Users/akankshitadash/Desktop/Gene Ontology/5points/RNASeq/Gene Names/AccNum'+str(idx+1)+'.txt','w') as f:
        for acc_num in acc_nums:
            f.write("%s\n" % acc_num)

750 750
937 937
1121 1121
853 853
1092 1092
607 607
863 863
961 961
563 563
632 632
582 582
663 663
716 716


In [29]:
dill.dump_session('../Weights/HMM_GaussianHMM_5points_RNASeq.db')